In [ ]:
!unzip AnimalDataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: AnimalDataset/ragno/OIP-eBblY9bQyUrRADbLuh7pMQHaE8.jpeg  
  inflating: AnimalDataset/ragno/OIP-Ebel8CWMjbPSxpiQs-hAKwHaFi.jpeg  
  inflating: AnimalDataset/ragno/OIP-EbErR8KNlYyBM9jlNrIGIAHaGK.jpeg  
  inflating: AnimalDataset/ragno/OIP-eBN9F0r9OL4COmgyEYGvaAEVEs.jpeg  
  inflating: AnimalDataset/ragno/OIP-EbpsFIp42kQg7B467MR5bQHaE7.jpeg  
  inflating: AnimalDataset/ragno/OIP-ecbjPxEXH_UreyOy1xjZlQHaFH.jpeg  
  inflating: AnimalDataset/ragno/OIP-EcBwehwooCBORsmW1hDq6gHaGm.jpeg  
  inflating: AnimalDataset/ragno/OIP-ECg-But0WowvNZcIb8QIeAHaEo.jpeg  
  inflating: AnimalDataset/ragno/OIP-EcIZuLV7tnHDYHDPO0wVdAHaE7.jpeg  
  inflating: AnimalDataset/ragno/OIP-ECk9akfPinT4wzOOcsYOqgHaE7.jpeg  
  inflating: AnimalDataset/ragno/OIP-EckToZ388WVMmxwQ8UlndgHaEK.jpeg  
  inflating: AnimalDataset/ragno/OIP-ecrlgjnUaHSlYUtyPmN5wwHaFS.jpeg  
  inflating: AnimalDataset/ragno/OIP-EcxdFy8-9UBZGwjSnNtTKAHaE8.jpeg  
  inflating: AnimalDataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import random_split, DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit DenseNet input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pre-trained model
])

In [ ]:
import os
data_dir = os.path.join(os.getcwd(), 'AnimalDataset')

In [ ]:
full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)


In [ ]:
# Define Split Ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate sizes
total_size = len(full_dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

# Split dataset
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model = models.densenet121(pretrained=True)

# Modify the classifier for 10 classes (Animals-10)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 10)

# Move model to GPU if available
model = model.to(device)

# ==========================
# 3. Define Loss & Optimizer
# ==========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ==========================
# 4. Train the Model
# ==========================
num_epochs = 10  # Change as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

# ==========================
# 5. Evaluate the Model
# ==========================
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print(f"Validation Accuracy: {val_acc:.2f}%")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 73.3MB/s]


Epoch [1/10], Loss: 0.5691, Train Acc: 81.45%
Epoch [2/10], Loss: 0.3440, Train Acc: 88.71%
Epoch [3/10], Loss: 0.2762, Train Acc: 91.00%
Epoch [4/10], Loss: 0.2269, Train Acc: 92.74%
Epoch [5/10], Loss: 0.1921, Train Acc: 93.71%
Epoch [6/10], Loss: 0.1700, Train Acc: 94.50%
Epoch [7/10], Loss: 0.1469, Train Acc: 95.20%
Epoch [8/10], Loss: 0.1302, Train Acc: 95.73%
Epoch [9/10], Loss: 0.1112, Train Acc: 96.24%
Epoch [10/10], Loss: 0.1098, Train Acc: 96.35%
Validation Accuracy: 90.64%


In [ ]:
# Define the path where you want to save the model
model_name = "densenet121_animals10.pth"

md = os.path.join(os.getcwd(), model_name)

# Save only the model weights (Recommended)
torch.save(model.state_dict(), md)

print("Model saved successfully!")

Model saved successfully!


In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.7 MB/s eta 0:00:00


In [ ]:
!pip install python-multipart

In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
from fastapi import FastAPI, File, UploadFile
from io import BytesIO
import json


app = FastAPI()

class_labels = ['dog', 'horse', 'elephant', 'butterfly', 'chicken', 'cat', 'cow', 'spider', 'sheep', 'squirrel']

# Load model architecture
model = models.densenet121(pretrained=False)

# Modify classifier to match Animals-10 (10 classes)
num_ftrs = model.classifier.in_features
model.classifier = torch.nn.Linear(num_ftrs, len(class_labels))

# Load trained weights
model.load_state_dict(torch.load("/content/densenet121_animals10.pth", map_location="cpu"))
model.eval()  # Set model to evaluation mode


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


def predict_image(image: Image.Image):
    """Preprocess the image and predict the class."""
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        predicted_class = output.argmax(1).item()
    return class_labels[predicted_class]

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    """API endpoint to classify an uploaded image."""
    image = Image.open(BytesIO(await file.read()))  # Read image from request
    prediction = predict_image(image)
    return {"prediction": prediction}


<ipython-input-5-072e37f2aa5e>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/densenet121_animals10.pth", map_location="cpu"))


In [6]:
import requests

url = "http://127.0.0.1:8000/predict/"
files = {"file": open("C:/Users/Chait/Desktop/API_Practise/cat.jpg", "rb")}

response = requests.post(url, files=files)
print(response.json())  # Should print {"prediction": "cane"} (or another class)


{'prediction': 'cat'}
